In [13]:
import numpy as np
import scipy
from tqdm import tqdm

In [14]:
class Node:
    def __init__(self, value: np.ndarray, parent: "Node | None"):
        """
        value: length N array of ints
        """

        self.value = value
        self.parent = parent

    def get_human_readable_value(self):
        # fmt: off
        value_to_base = [
            "AA", "CC", "GG", "TT",
            "AC", "AG", "AT", "CG",
            "CT", "GT", "CA", "GA",
            "TA", "GC", "TC", "TG",
        ]
        # fmt: on
        return "|".join([value_to_base[v] for v in self.value])

    def get_phy_value(self, human_readable=False):
        # fmt: off
        value_to_base = [
            "A", "C", "G", "T",
            # represent as unphased
            "M", "R", "W", "S", "Y", "K",
            "M", "R", "W", "S", "Y", "K",
        ]
        # fmt: on

        return "".join([value_to_base[v] for v in self.value])


class Simulate_GT16:
    pool_size = 20
    final_leaf_count = 20
    child_sample_count = 10
    dt = 1
    sim_steps = 10

    def __init__(self, exchangeability: np.ndarray, stationary: np.ndarray, N: int):
        """
        exchangeability: length 6
        stationary: length 16
        N: number of sites
        """

        self.pi = exchangeability
        self.stat = stationary
        self.N = N
        self.A = 16

        self.Q = self.get_Q()

        self.healthy_cell, self.leaf_cells = self.simulate()

    def get_Q(self):
        pi4 = np.repeat(self.pi, 4)

        # index helpers for Q matrix
        AA, CC, GG, TT, AC, AG, AT, CG, CT, GT, CA, GA, TA, GC, TC, TG = range(self.A)

        # fmt: off
        updates = [
            [AA, AC], [AC, CC], [AA, CA], [CA, CC], # A<->C
            [AA, AG], [AG, GG], [AA, GA], [GA, GG], # A<->G
            [AA, AT], [AT, TT], [AA, TA], [TA, TT], # A<->T
            [CC, CG], [CG, GG], [CC, GC], [GC, GG], # C<->G
            [CC, CT], [CT, TT], [CC, TC], [TC, TT], # C<->T
            [GG, GT], [GT, TT], [GG, TG], [TG, TT], # G<->T
        ]
        # fmt: on

        R = np.zeros((self.A, self.A))

        for update, pi_val in zip(updates, pi4):
            R[update[0], update[1]] = pi_val

        R = R + R.T

        y_q = R @ np.diag(self.stat)
        hyphens = np.sum(y_q, axis=1)
        Q = np.copy(y_q)
        np.fill_diagonal(Q, -hyphens)
        return Q

    def sample(self, probabilities: np.ndarray):
        """
        probabilities: length A
        """

        normalized_probabilities = probabilities / np.sum(probabilities)
        return np.random.choice(self.A, p=normalized_probabilities)

    def probabilities_to_cell(self, probabilities: np.ndarray, parent: Node | None):
        """
        probabilities: N x A
        """

        sampled_value = np.apply_along_axis(self.sample, 1, probabilities)
        return Node(sampled_value, parent)

    def evolve_cell(self, cell: Node):
        # expand length N array cell.value into N x A array of one-hot vectors
        value_one_hot = np.eye(self.A)[cell.value]

        probabilities = scipy.linalg.expm(self.Q * self.dt) @ value_one_hot.T  # A x N
        probabilities = probabilities.T  # N x A
        return self.probabilities_to_cell(probabilities, cell)

    def simulate(self) -> tuple[Node, list[Node]]:
        probabilities_across_sites = np.tile(self.stat, (self.N, 1))
        pool = [
            self.probabilities_to_cell(probabilities_across_sites, None)
            for _ in range(self.pool_size)
        ]

        healthy_cell = pool[0]

        for _ in tqdm(range(self.sim_steps)):
            new_pool = []

            for cell in pool:
                for _ in range(self.child_sample_count):
                    new_pool.append(self.evolve_cell(cell))

            pool = np.random.choice(new_pool, size=self.pool_size, replace=False)

        leaf_cells = np.random.choice(pool, size=self.final_leaf_count, replace=False)  # type: ignore
        return healthy_cell, list(leaf_cells)

    def save_to_dot_phy(self, path: str):
        total_cells = self.final_leaf_count + 1  # including healthy cell

        with open(path, "w") as f:
            f.write(f"{total_cells} {self.N}\n")
            f.write(f"Healthy    {self.healthy_cell.get_phy_value()}\n")

            for i, cell in enumerate(self.leaf_cells):
                f.write(f"CELL-{i}    {cell.get_phy_value()}\n")

In [15]:
exchangeability = np.array([0.001] * 6)
stationary = np.zeros(16)
stationary[0] = 1
N = 500

simulation = Simulate_GT16(exchangeability, stationary, N)

for cell in [simulation.healthy_cell] + simulation.leaf_cells:
    print(cell.get_human_readable_value())

path = "./data/cellphy_simulated_set.phy"

simulation.save_to_dot_phy(path)
print(f"Saved to {path}")

100%|██████████| 10/10 [00:16<00:00,  1.64s/it]

AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|AA|A